In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sklearn as sklearn
import seaborn as sns
import warnings
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import cv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,  StratifiedShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier, \
                            RandomForestClassifier,\
                            GradientBoostingRegressor,\
                            RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold,KFold
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:


#https://www.pythonpool.com/check-if-number-is-prime-in-python/

#Inputs: num - a number you want to check if it is prime 
#outputs: Ture or false epending if it is prime or not 
def isprime(num):
    for n in range(2,int(num**1/2)+1):
        if num%n==0:
            return False
    return True



#https://gist.github.com/iansan5653/1e306da9688d85934385b266e74f153a

#Inputs: c- takes in an int
#outputs: [b,a] - the closest two factors for the factorization of a given number
def calc_closest_factors(c: int):
    if c//1 != c:
        raise TypeError("c must be an integer.")
    if c>=5:
        if isprime(c)==True:
            c=c+1
    
    a, b, i = 1, c, 0
    while a < b:
        i += 1
        if c % i == 0:
            a = i
            b = c//a
    
    return [b, a]


#Purpose:Return and aggregated dataframe based on a given index,aggregation_value,and aggregation_calculation
#Input:
    #df - dataframe you wish to aggregate 
    #agg_value- value you wish to aggreate
    #index - group by index, can be a string or an array
    #agg_calc - aggreagte calculation you wish to preform, can be a string or an array
#output:
    #df - returns the new aggregate dataframe 
def get_aggCalc(df,agg_value,index,agg_calc):
        if type(index)==str:
            index = [index]
        df_filter = index.copy()
        df_filter.append(agg_value)
        df= df.groupby(index).agg({agg_value:agg_calc}).reset_index()
        if type(agg_calc)==list:
            df.columns = ['_'.join(col) for col in df.columns.values]
            df.columns = [col.split('_')[0] if col.split('_')[1]=='' else col for col in df.columns ]
        else:
            df = df.rename(columns = {agg_value:agg_value+"_"+agg_calc})
        return df
    
    

This one is a lot bigger. So for a lot of the questions on the homework, we are asked to plot two features, segmented out by another. This function does this for us, but with a lot of given flexibility. It is super helpful when testing out interactions between three variables. There are some adjustments we plan to make to make it more consistent (i.e., for the aggregate plots, we choose seaborn sometimes rather than matplotlib. This was becuase we found seaborn made it easier to fit our data as it didn't require special pivoting.)

So let's go over every input variable here and there expected typing.

* **df**: DataFrame - dataframe with all your data
* **decompose**: string - factor you wish to separate the relationship for. For example, if you wanted to see how bike rentals change by hour and season, you would set this to season.
* **xfeature**: string - this will be your x variable on the final plots.
* **yfeature**: string - this will be your y variable on the final plots. It's our dependent variable.
* **agg_choice**: string - which forms of aggregation do you want to use. Note this only supports one value right now, although it could be expanded to include a list.
* **Overlay**: Boolean - If you want to put all graphs on a single plot rather than individual subplots. Useful when trying to directly compare
* **plt_choice**: string - This will let the algorithm know what type of plot you want. For now, the following are supported.
 * scattter - scatter plot
 * scatter_agg - scatter plot, but taking an aggregate based on agg_choice to make only one point per xfeature value
 * line - line graph, this will automatically aggregate based on agg_choice, otherwise, it's too messy
 * bar - bar graph, this will automatically aggregate based on agg_choice otherwise, it's too messy


In [ ]:
def decompose_effect_on_two(df,decompose,xfeature,yfeature,plt_choice='scatter',agg_choice='mean',figsize_factor=1,overlay=False):
    x_list = df[decompose].unique()
    x_count = len(x_list)
    plt_x,plt_y = calc_closest_factors(x_count)
    if overlay == False:
        fig,ax = plt.subplots(plt_x,plt_y,figsize=(plt_y*4*figsize_factor, plt_x*4*figsize_factor))
        fig.tight_layout()
        plt_index = 1
        for x in x_list:
            plt.subplot(plt_x,plt_y,plt_index)
            df_split = df[df[decompose]==x]
            if plt_choice == 'scatter':
                plt.scatter(y=df_split[yfeature],x=df_split[xfeature])
            elif plt_choice =='box':
                sns.boxplot(x=df_split[xfeature],y=df_split[yfeature])
            else:
                agg_df = get_aggCalc(df,yfeature,xfeature,agg_choice)
                if plt_choice =='bar':
                    plt.bar(x = agg_df[xfeature],height=agg_df[yfeature+"_"+agg_choice])
                elif plt_choice =='line':
                    plt.plot(agg_df[yfeature+"_"+agg_choice])
                elif plt_choice =='scatter_agg':
                    plt.scatter(x = agg_df[xfeature],y=agg_df[yfeature+"_"+agg_choice])
            plt.xlabel(xfeature)
            plt.ylabel(yfeature)
            plt.title(x)
            plt_index = plt_index +1 
        fig.subplots_adjust(hspace=.5)  
    else:
        fig,ax = plt.subplots(figsize=(plt_y*4, plt_x*4))
        fig.tight_layout()
        if plt_choice == 'scatter':
            sns.scatterplot(data=df,y=yfeature,x=xfeature,hue=decompose)
            #plt.colorbar()
        elif plt_choice =='box':
                sns.boxplot(x=df[xfeature],y=df[yfeature],hue=df[decompose])
        else:
            agg_df = get_aggCalc(df,yfeature,index = [xfeature,decompose], agg_calc=agg_choice)
            if plt_choice =='bar':
                agg_df= agg_df.pivot(index=xfeature, columns=decompose, values=yfeature+"_"+agg_choice)
                agg_df.plot.bar(ax=ax)
            elif plt_choice =='line':
                sns.lineplot(data=agg_df,x=xfeature,y=yfeature+"_"+agg_choice,hue=decompose)
            elif plt_choice =='scatter_agg':
                 sns.scatterplot(data=agg_df,y=yfeature+"_"+agg_choice,x=xfeature,hue=decompose)
        plt.xlabel(xfeature)
        plt.ylabel(yfeature)
        plt.title('overlaid graph for relationship between '+ xfeature + ' on ' + yfeature +' separated by ' + decompose)    

Very similar to the above, just now with one feature (or an array of feature plotted one by one) against another. added figure_size option to increase size of plots

In [2]:
def plot_features(df,xfeature_array,yfeature_array,plt_choice='scatter',agg_choice='mean',figsize_factor=1,title=True):
    for yfeature in yfeature_array:
        x_list = xfeature_array
        x_count = len(x_list)
        plt_x,plt_y = calc_closest_factors(x_count)
        fig,ax = plt.subplots(plt_x,plt_y,figsize=(plt_y*4*figsize_factor, plt_x*4*figsize_factor))
        fig.tight_layout()
        if title==True:
            fig.suptitle('Plotting x features on '+ yfeature, fontsize=16)
        fig.subplots_adjust(top=0.8)
        plt_index=1
        for xfeature in x_list:
            plt.subplot(plt_x,plt_y,plt_index)
            if plt_choice == 'scatter':
                plt.scatter(y=df[yfeature],x=df[xfeature])
            elif plt_choice =='box':
                sns.boxplot(x=df[xfeature],y=df[yfeature])
            else:
                agg_df = get_aggCalc(df,yfeature,xfeature,agg_choice)
                if plt_choice =='bar':
                    plt.bar(x = agg_df[xfeature],height=agg_df[yfeature+"_"+agg_choice])
                elif plt_choice =='line':
                    plt.plot(agg_df[yfeature+"_"+agg_choice])
                elif plt_choice =='scatter_agg':
                    plt.scatter(x = agg_df[xfeature],y=agg_df[yfeature+"_"+agg_choice])
                
            plt.xlabel(xfeature)
            plt.ylabel(yfeature)
            plt_index = plt_index +1 
        fig.subplots_adjust(hspace=.5) 

## Code 

In [ ]:
#duplicating for sake of submission
bike_test = pd.read_table('Bike_test.csv',header=0,delimiter=',')
bike_train = pd.read_table('Bike_train.csv',header=0,delimiter=',')

#copy from above 
bike_train_modeling = bike_train.copy()
bike_test_modeling = bike_test.copy()

#Creating data time variable for feature exploration
bike_train_modeling['Date'] = pd.to_datetime(bike_train_modeling[['year','month','day']])
bike_test_modeling['Date'] = pd.to_datetime(bike_test_modeling[['year','month','day']])

#potential relaationship on weekends.  Non working day may tell us the same thing, but i think there are slight differences 
#that it's not bad to add
bike_train_modeling['weekday_number'] = bike_train_modeling['Date'].apply(lambda x: x.weekday())
bike_test_modeling['weekday_number'] = bike_test_modeling['Date'].apply(lambda x: x.weekday())


#debatable variables for cateogrical transformation: weather - it could be ordinal
categorical_variables = ['season','year','weather']

#Getting categorical Vairables
for cv in categorical_variables:
    bike_train_modeling[cv] = bike_train_modeling[cv].astype('category')
    bike_test_modeling[cv] = bike_test_modeling[cv].astype('category')
#move all ints to floats"
for col in bike_train_modeling.columns:
    if bike_train_modeling[col].dtype == 'int64':
        bike_train_modeling[col] = bike_train_modeling[col].astype('float64')
for col in bike_test_modeling.columns:
    if bike_test_modeling[col].dtype == 'int64':
        bike_test_modeling[col] = bike_test_modeling[col].astype('float64')
        
        
        
        
#run encoders
variableNames = categorical_variables
bike_train_temp_encoded = oneHot_vairables(bike_train_modeling,variableNames)
bike_test_temp_encoded = oneHot_vairables(bike_test_modeling,variableNames)
bike_train_temp_encoded = cycle_encode(bike_train_temp_encoded,['month','day','hour','weekday_number']) \
                            .drop(['month','day','weekday_number'],axis=1) #using hour will drop hour latter

bike_test_temp_encoded = cycle_encode(bike_test_temp_encoded,['month','day','hour','weekday_number']) \
                            .drop(['month','day','weekday_number'],axis=1)        

In [ ]:
def add_features(df):
    #See Question 1.C 
    df['peak_riding_time_workingDay'] = df[['hour','workingday']].apply(lambda df: 1 if (df['workingday'] == 1 and \
                                                        (7 <= df['hour'] <= 8  or 17 <= df['hour'] <= 18)) else 0,axis=1) 
    df['peak_riding_time_nonWorkingDay'] = df[['hour','workingday']].apply(lambda df: 1 if \
                                                        (df['workingday'] == 0 and 10 <= df['hour'] <= 19) else 0,axis=1)
    
    #we are going to remove atemp, so let's capture that boost we saw during exploration
    df['itFeelsHot'] = df['atemp'].apply(lambda x: 1 if x>=30 else 0)
    
    df['deadHour'] = df['hour'].apply(lambda x: 1 if x==4 else 0) #see above gaphs
    df['lowHumidity'] =df['humidity'].apply(lambda x: 1 if x<15 else 0)#see pdp
    df['logHumidity'] = df['humidity'].apply(lambda x: 0 if np.log(x+.001)<0 else np.log(x)) #quick log test
    df['LHum_HTemp'] = df[['humidity','temp']].apply(lambda df: 1 if (df.humidity>40 and df.temp<10) else 0,axis=1)#intercation  
    
    return df 
bike_train_temp_encoded = add_features(bike_train_temp_encoded)
bike_test_temp_encoded = add_features(bike_test_temp_encoded)
    
    
    

In [ ]:
drop_Features = ['daylabel', #little predictive power for daylabel plus is captured with month,year,day better
                 'windspeed',#already shown to have little importance in above graphs and confirmed with feature importance
                 'day_sin', #days play little into the ultimate prediction
                 'day_cos',
                 'holiday', #could be leading to worse prediction because there are more holidays in the latter half of the 
                            #month than. we could be undersestimating this power',
                 'weather_decent', #not predicitive
                 'weather_veryBad',
                 'hour', #encoded via clyical data don't want multi-collinearity issues
                 'atemp', #multicollinearity issues
                 'Date',
                 'lowHumidity', #not important
                 'logHumidity',
                 'LHum_HTemp' #shap Value gave some insights there may be a spike when it's low humidty and high temperature
                             # but couldn't find it
                ]

In [ ]:
bike_test_temp_encoded = bike_test_temp_encoded.drop(drop_Features,axis=1)
X = bike_train_temp_encoded.drop(['count'],axis=1)
X = X.drop(drop_Features,axis=1)
y = bike_train_temp_encoded['count']


Hyperparamter tuning 

In [ ]:
params = {
 'learning_rate' : [.05,.10,.15,.20,.25,.30],
 'n_estimators' : [50,100,200,1000],
 'max_depth' : [3,4,5,6,7,8,9,10],
 'min_child_weight' : [1,2,3,4,5,6],
 'gamma': [0,.1,.2,.3,.4,.5,.6],
 'colsample_bytree' : [.3,.4, .5,.6,.7 ]

}

gb_tree = XGBRegressor(seed = 1)

rs_cv = RandomizedSearchCV(estimator=gb_tree,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         #verbose=1,
                         n_iter= 1000)#change depending on how many fits we want to try. 
rs_cv.fit(X, y)
print("Best parameters:", rs_cv.best_params_)
print('Best MSE '+ str(rs_cv.best_score_))
best_params_ = rs_cv.best_params_

Fit

In [ ]:
best_params = best_params_
gb_tree = XGBRegressor(seed = 1,)
gb_tree.set_params(**best_params)
gb_tree.fit(X,y)

Feature Importance 

In [ ]:
gb_feature_importance = gb_tree.feature_importances_
sorted_index = np.argsort(gb_feature_importance)
pos = np.arange(sorted_index.shape[0]) + 0.5
fig, ax = plt.subplots(figsize=(12, 20))
plt.barh(pos, gb_feature_importance[sorted_index], align="center")
plt.yticks(pos, np.array(X.columns)[sorted_index]);
plt.title('feature importance for gradient boosting');

Predict 

In [ ]:
y_pred_test = gb_tree.predict(bike_test_temp_encoded)
df_final = pd.DataFrame({'count':y_pred_test})
df_final = df_final.reset_index()
df_final['index'] = df_final['index'] + 1 #correct for difference in indexing 
df_final.rename(columns={'index':'Id'},inplace=True)

#Note: I really want to round this, it just preformed worse. 
#I hate giving answers that don't make sense in terms of the problem
df_final['count'] = df_final['count'].apply(lambda x: 0 if x<0 else x) 
df_final.to_csv('sampleSubmission.csv',index=False)